In [1]:
import pandas as pd
import numpy as np
from joblib import dump

In [2]:
data = pd.read_excel("komentar/data_clean_from_dataset.xlsx")
komentar = data.komentar.tolist()
label = np.array(data.label.tolist())
y = label
data.head()

,Unnamed: 0,no,label,komentar
0,0,0,non spam,waduh mara ken ae mbak via ni kirim balikpapan...
1,1,1,spam,awass akun adu domba min tu org suka nyinyirin...
2,2,2,non spam,begini kalo ga biasa lipsing sekali lipsing ku...
3,3,3,non spam,nitijen mha bebar bkin mlu 2 ajh kliaan apa sd...
4,4,4,spam,maantaap banggeet kaak ssetelaah akuu pakee ge...


# Pemodelan

## Pemodelan TF-IDF

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(komentar)
save = vectorizer.fit(komentar)
dump(save, "model/tfidf.w")

#print(vectorizer.get_feature_names())
print(X.shape)

(4948, 14795)


## Pemodelan MNB 

In [1]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()

# Pelatihan dan Validasi Model

In [5]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score

skf = StratifiedKFold(n_splits=10)
skf.get_n_splits(X, y)

start = 0.001
stop = 1
step = start
alpha_list = np.arange(start,stop+step,step)
akurasi = list()

ix = 0
for alpha in alpha_list:
    clf.set_params(alpha=alpha)
    temp_akurasi = list()
    
    for train_index, test_index in skf.split(X, y):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        clf.fit(X_train,y_train)
        
        y_pred = clf.predict(X_test)
        temp_akurasi.append(accuracy_score(y_test, y_pred))
        
    rerata_akurasi = sum(temp_akurasi)/len(temp_akurasi)
    akurasi.append(rerata_akurasi)
    
    ix +=1
    if ix%100 !=0:
        print(".", end="")
    else:
        print("")

        pass

ditt = {
    "alpha":alpha_list,
    "akurasi":akurasi
}

index = akurasi.index(max(akurasi))
acc = pd.DataFrame.from_dict(ditt)
acc

...................................................................................................
...................................................................................................
...................................................................................................
...................................................................................................
...................................................................................................
...................................................................................................
...................................................................................................
...................................................................................................
...................................................................................................
...................................................................................................


,alpha,akurasi
0,0.001,0.944834
1,0.002,0.946047
2,0.003,0.947058
3,0.004,0.948474
4,0.005,0.948676
5,0.006,0.949283
6,0.007,0.949687
7,0.008,0.949283
8,0.009,0.949283
9,0.010,0.949890


In [6]:
print(alpha_list[index])
print(akurasi[index])

0.20600000000000002
0.9561642941099647


# Menyimpan Model

In [7]:
clf.set_params(alpha=alpha_list[index])
clf.fit(X, y)

dump(clf, "model/mnb.m")

['model/mnb.m']